In [ ]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

In [ ]:
import mmcv
# Check MMRotate installation
import mmrotate
print(mmrotate.__version__)

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

In [ ]:
from mmrotate.registry import DATASETS
from mmrotate.datasets.dota import DOTADataset


@DATASETS.register_module()
class PEdataset(DOTADataset):
    """Pedicle for detection."""
    METAINFO = {
        'classes':
        ('pedicle',),
        # palette is a list of color tuples, which is used for visualization.
        'palette': [(165, 42, 42)]
    }

In [ ]:
from mmengine import Config
cfg = Config.fromfile('mmrotate/configs/redet/redet-le90_re50_refpn_1x_dota.py')

In [ ]:
# Modify dataset type and path
cfg.dataset_type = 'PEdataset'
cfg.data_root = '.'


cfg.train_dataloader.dataset.type = 'PEdataset'
cfg.train_dataloader.dataset.data_root = './pd_val'
cfg.train_dataloader.dataset.ann_file = 'tr_pd_label'
cfg.train_dataloader.dataset.data_prefix.img_path = 'tr_pd_img'
cfg.train_dataloader.batch_size=2
cfg.train_dataloader.num_workers=0
cfg.train_dataloader.persistent_workers=False


cfg.val_dataloader.dataset.type = 'PEdataset'
# cfg.val_dataloader.dataset.data_root = 'pedicle_data/'
cfg.val_dataloader.dataset.data_root = './pd_val'
cfg.val_dataloader.dataset.ann_file = 'val_pd_label'
cfg.val_dataloader.dataset.data_prefix.img_path = 'val_pd_img'
cfg.val_dataloader.batch_size=10 # Increasing the batch size for faster validation
cfg.val_dataloader.num_workers=0
cfg.val_dataloader.persistent_workers=False


# AdamW optimizer
cfg.optim_wrapper = dict(
    clip_grad=dict(max_norm=25, norm_type=2),
    optimizer=dict(lr=1e-7,type='AdamW', weight_decay=1e-5),
    type='OptimWrapper')


cfg.test_dataloader = cfg.val_dataloader

cfg.model.roi_head.bbox_head[0].num_classes = 1
cfg.model.roi_head.bbox_head[1].num_classes = 1

# cfg.model.backbone.init_cfg.checkpoint='redet_re50_fpn_1x_dota_ms_rr_le90-fc9217b5.pth'
cfg.model.backbone.init_cfg.checkpoint='re_resnet50_c8_batch256-25b16846.pth'    # 초기 모델 적용, 학습을 새로 시작할 때
cfg.load_from = 'PD_ReDet_RT4_LR5e-7_G7184_D7487_R0.961_A0.905_e23.pth'

cfg.work_dir = './pd_val'

cfg.train_cfg.max_epochs = 1
cfg.train_cfg.val_interval = 1

cfg.default_hooks.logger.interval = 1

# Set seed to facilitate reproducing the result
cfg['randomness'] = dict(seed=3407)

print(f'Config:\n{cfg.pretty_text}')

In [ ]:
from mmengine.runner import Runner

runner = Runner.from_cfg(cfg)

In [ ]:
runner.val()